# Process raw file from facebook

In [23]:
# Convert the json to correct format to avoid \u00e2 blablabla
import re
import json

with open('chloe/message_1.json') as f:
    data = json.load(f)


In [24]:
updated_json = []
for subdata in data:
    if u'content' in subdata:
        s = subdata[u'content']
        subdata[u'content'] = re.sub(r'[\xc2-\xf4][\x80-\xbf]+',lambda m: m.group(0).encode('latin1').decode('utf8'),s)
    updated_json.append(subdata)
    
with open('chloe/corrected_message.json', 'w') as f:
    json.dump(data, f)

# Load Corrected file

In [1]:
import pandas as pd
import numpy as np

In [41]:
# load message to dataframe
chloe = pd.read_json('chloe/corrected_message.json', encoding="utf-8").drop(columns=['type', 'is_unsent', 'photos', 'call_duration', 'reactions', 'videos', 'sticker'])
# let's apply some filters
chloe['content'] = chloe['content'].str.lower()
chloe = chloe.dropna()
chloe = chloe.reset_index(drop=True)

In [42]:
chloe

,sender_name,timestamp_ms,content
0,Chloe Berthet,2021-10-20 21:52:35.807,bah l’épice est tombée depuis en haut sur l’as...
1,Sven Borden,2021-10-20 21:51:37.781,how
2,Sven Borden,2021-10-20 21:51:35.500,oh non
3,Chloe Berthet,2021-10-20 21:38:46.503,oupsi
4,Chloe Berthet,2021-10-20 21:38:44.801,hum
...,...,...,...
3506,Chloe Berthet,2021-09-21 08:02:31.056,vous voulez aller ou ?
3507,Chloe Berthet,2021-09-21 08:02:25.475,mais faut que je rentre à aarau après donc je ...
3508,Chloe Berthet,2021-09-21 08:01:48.815,oh c’est tentant
3509,Sven Borden,2021-09-21 08:01:44.261,enfin pas tard


# Compute the nearest string from messages and find the best answer

In [43]:
from fuzzywuzzy import fuzz, process
import Levenshtein as lev


In [55]:
def lev_distance(w1, list_w2):
    distance = []
    for w2 in list_w2:
        distance.append(lev.distance(w1, w2))
    return distance

def lev_ratio(w1, list_w2):
    ratio = []
    for w2 in list_w2:
        ratio.append(fuzz.ratio(w1, w2))
    return ratio

def lev_partial_ratio(w1, list_w2):
    ratio = []
    for w2 in list_w2:
        ratio.append(fuzz.partial_ratio(w1, w2))
    return ratio

def lev_token_sort_ratio(w1, list_w2):
    ratio = []
    for w2 in list_w2:
        ratio.append(fuzz.token_sort_ratio(w1, w2))
    return ratio

def lev_token_set_ratio(w1, list_w2):
    ratio = []
    for w2 in list_w2:
        ratio.append(fuzz.token_set_ratio(w1, w2))
    return ratio

def lev_process(w1, list_w2):
    return process.extractOne(w1, list_w2)

In [77]:
def get_ind_answer(ind):
    if ind <= 0:
        return 0
    while ind > 0:
        ind -= 1
        if chloe.iloc[ind]['sender_name'] == 'Sven Borden':
            return ind

In [79]:
word = "Est-ce que tu as vu ce tiktok"
word = word.lower()

# Select only chloe text
chloe_only = chloe[chloe['sender_name'] == 'Chloe Berthet']['content']
ratios = lev_process(word, chloe_only.to_list())
print(f'process returned {ratios}')
ind = chloe.index[chloe['content'] == ratios[0]].to_list()
print(f'INDEXES {ind} CONTENT {chloe.iloc[ind]}')

answers = []
for i in ind:
    answers.append(chloe.iloc[get_ind_answer(i)]['content'])
# Sort the 5 best scores and get indexes
print(answers)


process returned ('bah l’épice est tombée depuis en haut sur l’assiette 😂', 86)
INDEXES [0] CONTENT      sender_name            timestamp_ms  \
0  Chloe Berthet 2021-10-20 21:52:35.807   

                                             content  
0  bah l’épice est tombée depuis en haut sur l’as...  
['bah l’épice est tombée depuis en haut sur l’assiette 😂']


In [78]:
ratios[1]

86